Task:
1. For each province download respective data for VH in csv format (implement as a separate function which accepts province ID as an argument; name of file to which data will be saved must contain date and time of download; sources: https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/vh_adminMean.php?type=Province_Weekly_MeanPlot https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/vh_adminMeanByCrop.php?type=Province_Weekly_PAreaPlot).
1. Read and merge aformentioned csv files to pandas dataframe (implement as a separate function which accepts path to the csv files as an arguemnt).
1. Implement a function which will replace all province IDs with corresponding province Names.
1. Select from pandas dataframe following information (implement as a separate function):
    1. Max and min value of VHI siries for particular province and over particular year
    1. For a particular province all years when there was an extreme drought which affected more the certain percentage of area
    1. For a particular province all years when there was an mild drought which affected more the certain percentage of area

In [1]:
import urllib

from datetime import datetime

import glob

import pandas as pd

Let us implement function that will download data from http link and save it in csv format. 

In [2]:
def get_csv_province(province_ID, url, file_name_prefix):
    
    today = datetime.now().strftime("%d_%m_%Y_%H_%M_%S") # get the date and time of downloading

    url_parsed = list(urllib.parse.urlparse(url)) # parse url convert resulting named tuple to list
    
    query = url_parsed[4].split('&',1) # split query part of parsed url into atomic queries
    query[0] = query[0].split('=',1)[0] + str(province_ID) # modify atomic query provinceID

    url_parsed[4] = '&'.join(query) # reconstruct query part of parsed url

    url = urllib.parse.urlunparse(url_parsed) # reconstruct url

    with urllib.request.urlopen(url) as f: # get data as string through url
        data_str = f.read().decode("ASCII")

    data_str = data_str.split("<br>\n<tt><pre>")[1] # clean up string to match csv format
    data_str = data_str.replace("</pre></tt>","")
    data_str = data_str.replace(", ",",")
    
    # print(data_str)
    
    file_name = "{}_{}_{}.csv".format(file_name_prefix, province_ID, today) # generate a file name
    
    with open(file_name, "w") as f: # open an existing csv file with corresponding name or create new one with as handles all exceptions
        f.write(data_str) # overwrite or write csv string to aformentioned file
    
    return file_name

In [3]:
url_for_dataVHI = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID=1&country=UKR&yearlyTag=Weekly&type=Mean&TagCropland=crop&year1=1982&year2=2022"
file_name_prefix_for_dataVHI = "dataVHI"

for i in range(1, 28):
    str_file_name = get_csv_province(i, url_for_dataVHI, file_name_prefix_for_dataVHI) # download data VHI for each province

# headers = ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI"]
# df = pd.read_csv(str_file_name, header = 0, names = headers, index_col = False) # force pandas.read_csv() not to take first column as an index
# df.head(5)

In [4]:
url_for_dataVHIarea = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID=1&country=UKR&yearlyTag=Weekly&type=Parea_VHI&TagCropland=crop&year1=1982&year2=2022"
file_name_prefix_for_dataVHIarea = "dataVHIarea"

for i in range(1, 28):
    str_file_name = get_csv_province(i, url_for_dataVHIarea, file_name_prefix_for_dataVHIarea) # download data VHI area for each province

# df = pd.read_csv(str_file_name, header = 1, names = headers, index_col = False) # force pandas.read_csv() not to take first column as an index
# df.head(5)

Let us implement a separate function which accepts path to the csv files as an arguemnt and reads them to pandas dataframe.

In [5]:
def csv_files_to_df(path_prefix, file_name_prefix, headers):

    path = glob.glob("{}{}_{}_*.csv".format(path_prefix,file_name_prefix,1)) # get all csv files for particular province
    path.sort() # sort all pathes to get the latest downloaded csv file for particualr province
    # print(path)
    
    df = pd.read_csv(path[-1], header = None, names = headers, index_col = False) # read designated csv to resulting dataframe
    
    df["province"] = 1 # add column province to dataframe and fill it with corresponding value
    
    # df.head(5)
    
    for provinceID in range(2, 28): # loop through provinces for which csv file will be read 
        
        path = glob.glob("{}{}_{}_*.csv".format(path_prefix,file_name_prefix,provinceID)) # get all csv files for particular province 
        path.sort() # sort all pathes to get the latest downloaded csv file for particualr province
        # print(path)
        
        df_temp = pd.read_csv(path[-1], header = None, names = headers, index_col = False) # read designated csv to temporary dataframe  
        
        df_temp["province"] = provinceID # add column province to dataframe and fill it with corresponding value
        
        # df_temp.head(5)
        
        df = pd.concat([df,df_temp]) # concatenate temporary dataframe and resulting dataframe
    
    return df 

In [6]:
df_dataVHI = csv_files_to_df("./", "dataVHI",  ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI"]) # read all csv files to dataframe
df_dataVHI.head(5)

,year,week,SMN,SMT,VCI,TCI,VHI,province
0,1982,1,0.056,259.12,45.72,50.68,48.20,1
1,1982,2,0.056,260.63,46.97,44.53,45.75,1
2,1982,3,0.055,261.81,46.34,40.64,43.49,1
3,1982,4,0.053,263.17,43.47,37.35,40.41,1
4,1982,5,0.050,263.99,38.42,38.13,38.28,1


In [14]:
headers = ["year", "week"]
for i in range(0, 20):
        headers.append("VHI_range_{}_to_{}".format(i * 5, i * 5 + 5))
headers.append("VHI_range_100_to_inf")

df_dataVHIarea = csv_files_to_df("./", "dataVHIarea",  headers) # read all csv files to dataframe
df_dataVHIarea.head(5)

,year,week,VHI_range_0_to_5,VHI_range_5_to_10,VHI_range_10_to_15,VHI_range_15_to_20,VHI_range_20_to_25,VHI_range_25_to_30,VHI_range_30_to_35,VHI_range_35_to_40,...,VHI_range_60_to_65,VHI_range_65_to_70,VHI_range_70_to_75,VHI_range_75_to_80,VHI_range_80_to_85,VHI_range_85_to_90,VHI_range_90_to_95,VHI_range_95_to_100,VHI_range_100_to_inf,province
0,1982,1,0.0,0.01,0.06,0.35,1.88,5.16,8.93,12.35,...,8.15,5.22,3.23,1.58,0.49,0.14,0.02,0.0,0.0,1
1,1982,2,0.0,0.01,0.06,0.37,2.19,6.35,11.08,14.22,...,6.17,3.57,1.88,0.75,0.18,0.04,0.00,0.0,0.0,1
2,1982,3,0.0,0.00,0.03,0.29,2.29,7.44,13.12,16.46,...,4.04,1.83,0.71,0.24,0.04,0.01,0.00,0.0,0.0,1
3,1982,4,0.0,0.00,0.02,0.24,2.78,9.82,17.80,20.80,...,1.75,0.51,0.17,0.06,0.01,0.00,0.00,0.0,0.0,1
4,1982,5,0.0,0.00,0.01,0.18,2.99,12.53,22.66,23.43,...,0.78,0.21,0.03,0.03,0.00,0.00,0.00,0.0,0.0,1


In [17]:
df = df_dataVHIarea[headers[2:]]
df = pd.concat([df_dataVHI, df], axis = 1)

df.head(5)
# print(df.columns.tolist())

,year,week,SMN,SMT,VCI,TCI,VHI,province,VHI_range_0_to_5,VHI_range_5_to_10,...,VHI_range_55_to_60,VHI_range_60_to_65,VHI_range_65_to_70,VHI_range_70_to_75,VHI_range_75_to_80,VHI_range_80_to_85,VHI_range_85_to_90,VHI_range_90_to_95,VHI_range_95_to_100,VHI_range_100_to_inf
0,1982,1,0.056,259.12,45.72,50.68,48.20,1,0.0,0.01,...,11.56,8.15,5.22,3.23,1.58,0.49,0.14,0.02,0.0,0.0
1,1982,2,0.056,260.63,46.97,44.53,45.75,1,0.0,0.01,...,10.00,6.17,3.57,1.88,0.75,0.18,0.04,0.00,0.0,0.0
2,1982,3,0.055,261.81,46.34,40.64,43.49,1,0.0,0.00,...,8.01,4.04,1.83,0.71,0.24,0.04,0.01,0.00,0.0,0.0
3,1982,4,0.053,263.17,43.47,37.35,40.41,1,0.0,0.00,...,4.72,1.75,0.51,0.17,0.06,0.01,0.00,0.00,0.0,0.0
4,1982,5,0.050,263.99,38.42,38.13,38.28,1,0.0,0.00,...,2.64,0.78,0.21,0.03,0.03,0.00,0.00,0.00,0.0,0.0


In [18]:
df = df.drop(df.loc[df["VHI"] == -1].index) # drop all rows with nan VHI

df.tail(5)

,year,week,SMN,SMT,VCI,TCI,VHI,province,VHI_range_0_to_5,VHI_range_5_to_10,...,VHI_range_55_to_60,VHI_range_60_to_65,VHI_range_65_to_70,VHI_range_70_to_75,VHI_range_75_to_80,VHI_range_80_to_85,VHI_range_85_to_90,VHI_range_90_to_95,VHI_range_95_to_100,VHI_range_100_to_inf
2122,2022,43,0.191,282.85,57.07,26.13,41.60,27,0.49,0.97,...,7.67,5.60,3.50,1.57,0.74,0.22,0.07,0.02,0.01,0.00
2123,2022,44,0.169,280.98,55.72,18.98,37.34,27,0.66,1.43,...,5.34,3.12,1.46,0.58,0.23,0.10,0.03,0.02,0.01,0.00
2124,2022,45,0.159,279.06,59.01,16.06,37.53,27,0.68,1.44,...,5.31,2.85,1.32,0.59,0.33,0.14,0.08,0.04,0.02,0.01
2125,2022,46,0.141,276.62,58.43,17.25,37.84,27,0.52,1.50,...,4.75,2.85,1.56,0.85,0.45,0.29,0.15,0.09,0.04,0.04
2126,2022,47,0.124,273.48,57.56,22.52,40.04,27,0.28,1.09,...,5.09,3.63,2.37,1.58,0.93,0.60,0.41,0.22,0.13,0.12


Let us implement a function which will replace all province IDs with corresponding province Names.

In [19]:
def replace_province_ID_name(df):
    # create dictionary for column province that contatits for each provinceID its corresponding name
    province_ID_name = {"province": {1: "Cherkasy", 2: "Chernihiv", 3: "Chernivtsi", 4: "Crimea", 5: "Dnipropetrovs`k",
                                     6: "Donets`k", 7: "Ivano-Frankivs`k", 8: "Kharkiv", 9: "Kherson", 10: "Khmel`nyts`kyy",
                                     11: "Kiev", 12: "Kiev City", 13: "Kirovohrad", 14: "Luhans`k", 15: "L`viv", 16: "Mykolaiv",
                                     17: "Odessa", 18: "Poltava", 19: "Rivne", 20: "Sevastopol`", 21: "Sumy", 22: "Ternopil`",
                                     23: "Transcarpatl", 24: "Vinnytsa", 25: "Volyn", 26: "Zaporizhzhya", 27: "Zhytomyr"}}
    return df.replace(province_ID_name) # replace all provinceID with corresponding name using replace method from pandas

In [20]:
df = replace_province_ID_name(df) # replace in column province provinceID with provinceName
df.tail(5)

,year,week,SMN,SMT,VCI,TCI,VHI,province,VHI_range_0_to_5,VHI_range_5_to_10,...,VHI_range_55_to_60,VHI_range_60_to_65,VHI_range_65_to_70,VHI_range_70_to_75,VHI_range_75_to_80,VHI_range_80_to_85,VHI_range_85_to_90,VHI_range_90_to_95,VHI_range_95_to_100,VHI_range_100_to_inf
2122,2022,43,0.191,282.85,57.07,26.13,41.60,Zhytomyr,0.49,0.97,...,7.67,5.60,3.50,1.57,0.74,0.22,0.07,0.02,0.01,0.00
2123,2022,44,0.169,280.98,55.72,18.98,37.34,Zhytomyr,0.66,1.43,...,5.34,3.12,1.46,0.58,0.23,0.10,0.03,0.02,0.01,0.00
2124,2022,45,0.159,279.06,59.01,16.06,37.53,Zhytomyr,0.68,1.44,...,5.31,2.85,1.32,0.59,0.33,0.14,0.08,0.04,0.02,0.01
2125,2022,46,0.141,276.62,58.43,17.25,37.84,Zhytomyr,0.52,1.50,...,4.75,2.85,1.56,0.85,0.45,0.29,0.15,0.09,0.04,0.04
2126,2022,47,0.124,273.48,57.56,22.52,40.04,Zhytomyr,0.28,1.09,...,5.09,3.63,2.37,1.58,0.93,0.60,0.41,0.22,0.13,0.12


Let us implement separate function that selects from pandas dataframe max and min value of VHI siries for particular province and over particular year.

In [21]:
def VHI_province_year_extr(provinceName, year, df):
    province_year_VHI_max = df[(df["province"] == provinceName) & (df["year"] == year)]["VHI"].max() # get max value of VHI for given province and over given year
    province_year_VHI_min = df[(df["province"] == provinceName) & (df["year"] == year)]["VHI"].min() # get min value of VHI for given province and over given year
    return (province_year_VHI_min, province_year_VHI_max)

In [22]:
provinceName = "Cherkasy" # set province
year = 1982 # set year

extr = VHI_province_year_extr(provinceName, year, df)

print("max value of VHI for province {} over year {} is {}".format(provinceName, year, extr[1]))
print("min value of VHI for province {} over year {} is {}".format(provinceName, year, extr[0]))

max value of VHI for province Cherkasy over year 1982 is 62.35
min value of VHI for province Cherkasy over year 1982 is 28.87


Let us implement separate function that selects from pandas dataframe for a particular province all years when there was an extreme drought which affected more the certain percentage of area.

In [32]:
def get_years_extreme_drought_province_area(provinceName, area_perc, df):
    df = df[(df["province"] == provinceName) & (df["VHI_range_0_to_5"] >= area_perc)]
    return df["year"].unique()

In [33]:
provinceName = "Cherkasy" # set province
area_perc = 0 # set year

get_years_extreme_drought_province_area(provinceName, area_perc, df)

array([1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

Let us implement separate function that selects from pandas dataframe for a particular province all years when there was an mild drought which affected more the certain percentage of area.

In [34]:
def get_years_extreme_drought_province_area(provinceName, area_perc, df):
    df = df[(df["province"] == provinceName) & ((df["VHI_range_5_to_10"] + df["VHI_range_10_to_15"]) >= area_perc)]
    return df["year"].unique()

In [35]:
provinceName = "Cherkasy" # set province
area_perc = 0 # set year

get_years_extreme_drought_province_area(provinceName, area_perc, df)

array([1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)